<a href="https://colab.research.google.com/github/quirkyabhi/EEG-EMOTIONAL-ANALYSIS/blob/master/codes/cnn_pli_final_new_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras import backend as K
K.set_image_dim_ordering('tf')
import os
import tensorflow as tf
import numpy as np
import scipy.io
import time
import datetime
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold

from scipy.interpolate import griddata
from sklearn.preprocessing import scale
from functools import reduce
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense, Conv2D, MaxPooling2D, Conv1D, MaxPool1D
from keras.models import Sequential
from keras.layers import Dropout, Input, BatchNormalization
from sklearn.metrics import confusion_matrix, accuracy_score
# from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
# import plotly.graph_objs as go
# from matplotlib.pyplot import cm
# from keras.models import Model
import numpy as np

import keras
# import h5py
from keras.utils import to_categorical
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [0]:
def load(datafile):
    
    try:
        dataMat = scipy.io.loadmat('images_average.mat', mat_dtype=True)
        print("Data loading complete. Shape is %r" % (dataMat['images_plv'].shape))
    except:
        try:
            dataMat = pd.read_csv(datafile,  index=False, header= None)
        except:
            dataMat=pd.read_excel(datafile, index=False, header= None)
        
    try:
        return dataMat['images_plv']
    except:
        return dataMat

In [0]:
def reformatInput(data, labels):
    indices = np.random.permutation(147200)

    trainIndices = [indices[:int(147200*.8)]]
    validIndices = [indices[int(147200*.8):]]

    if data.ndim == 3:
        return [(data[trainIndices], np.squeeze(labels[trainIndices]).astype(np.int32)),
                (data[validIndices], np.squeeze(labels[validIndices]).astype(np.int32))]
#                 (data[testIndices], np.squeeze(labels[testIndices]).astype(np.int32))]
    elif data.ndim == 5:
        return [(data[:, trainIndices], np.squeeze(labels[trainIndices]).astype(np.int32)),
                (data[:, validIndices], np.squeeze(labels[validIndices]).astype(np.int32))]

In [0]:
def make_matrix(df):
#     mat=np.array(df[1,:])
    return df.values

In [0]:
# df=pd.read_csv('plv_csv.csv', header=None)

In [0]:
df=pd.read_csv('drive/My Drive/EEG/PLV_final_dist2.csv',  header=None)
ldf=pd.read_csv('drive/My Drive/EEG/arousal_label_total.csv', header= None)
mat=make_matrix(df)
ldf=make_matrix(ldf)
ldf=np.asarray(ldf)
mat.shape

(147200, 1024)

In [0]:
finalmat=[]
ldf.shape


(147200, 1)

In [0]:
for i in range(len(mat)):
    finalmat.append(mat[i,:].reshape(32,32))
train=np.asarray(finalmat)

In [0]:
(X_train, y_train), (X_test, y_test) = reformatInput(train, ldf)
# y_train=to_categorical(y_train)
# y_test=to_categorical(y_test)
X_train = X_train.astype(float).reshape(117760,32,32,1)
X_test = X_test.astype(float).reshape(147200-117760,32,32,1)
y_train

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if __name__ == '__main__':


array([0, 0, 1, ..., 1, 1, 0], dtype=int32)

In [0]:
from keras import backend as K
K.set_image_dim_ordering('tf')
def make_model():
  
  num_category = 2
  # t_train=y_train
  # y_test=y_val
  # y_train = keras.utils.to_categorical(y_train, num_category)
  # y_test = keras.utils.to_categorical(y_val, num_category)
  model = Sequential()
  #convolutional layer with rectified linear unit activation
  model.add(Conv2D(32, kernel_size=3,activation='tanh',input_shape=(32,32,1), ))
  #32 convolution filters used each of size 3x3
  #again
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())
#   model.add(Dropout(0.2))
  model.add(Conv2D(64, 3, activation=keras.activations.tanh))
  model.add(Conv2D(128, kernel_size=3,activation='tanh'))
#   #64 convolution filters used each of size 3x3
#   #choose the best features via pooling
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())
#   model.add(Dropout(0.25))
#   model.add(Conv2D(128, kernel_size=3,activation='relu'))
#   model.add(MaxPooling2D(pool_size=(2, 2)))
#   model.add(BatchNormalization())
#   model.add(Dropout(0.2))
  # randomly turn neurons on and off to improve convergence
  # model.add(Dropout(0.25))
  # model.add(Conv2D(256, (3, 3), activation='relu'))
  # #64 convolution filters used each of size 3x3
  # #choose the best features via pooling
  # model.add(MaxPooling2D(pool_size=(2, 2)))
  # model.add(BatchNormalization())
  # model.add(Dropout(0.2))
  # model.add(Conv2D(512, kernel_size=(3, 3),
  #                  activation='relu'))
  # model.add(MaxPooling2D(pool_size=(2, 2)))
  # model.add(BatchNormalization())
  # model.add(Dropout(0.1))
  # # randomly turn neurons on and off to improve convergence
  # model.add(Dropout(0.25))
  # # flatten since too many dimensions, we only want a classification output
  model.add(Flatten())
  #fully connected to get all relevant data
  model.add(Dense(128, activation='tanh'))
  #one more dropout for convergence' sake :) 
#   model.add(Dropout(0.5))
  #output a softmax to squash the matrix into output probabilities
  model.add(Dense(2, activation='softplus'))
  print(model.summary())
#   model.compile(loss=keras.losses.binary_crossentropy,
#               optimizer=keras.optimizers.Adam(.0001),
#               metrics=['accuracy'])
  return model
model=make_model()

W0720 08:37:02.800003 139810037974912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0720 08:37:02.841671 139810037974912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0720 08:37:02.850597 139810037974912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0720 08:37:02.866967 139810037974912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0720 08:37:02.883432 139810037974912 deprecation_wrapp

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 15, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 128)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 5, 5, 128)         512       
__________

In [0]:
# print(y_train.shape)
# model.compile(loss=keras.losses.sparse_categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])
# batch_size = 256
# num_epoch = 100
# #model training
# model_log = model.fit(X_train, y_train,
#           batch_size=batch_size,
#           epochs=num_epoch,
#           verbose=1,
#           validation_data=(X_test, y_test))

In [0]:
print(y_train.shape)
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.SGD(.00001, decay=10**-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])
batch_size = 256
num_epoch = 1000
#model training
model_log = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=(X_test, y_test))

(117760,)
Train on 117760 samples, validate on 29440 samples
Epoch 1/1000
117760/117760 [==============================] - 10s 83us/step - loss: 0.6719 - acc: 0.5982 - val_loss: 0.6756 - val_acc: 0.5910
Epoch 2/1000
117760/117760 [==============================] - 9s 78us/step - loss: 0.6719 - acc: 0.5984 - val_loss: 0.6757 - val_acc: 0.5908
Epoch 3/1000
117760/117760 [==============================] - 9s 80us/step - loss: 0.6718 - acc: 0.5984 - val_loss: 0.6760 - val_acc: 0.5908
Epoch 4/1000
117760/117760 [==============================] - 9s 80us/step - loss: 0.6718 - acc: 0.5984 - val_loss: 0.6756 - val_acc: 0.5913
Epoch 5/1000
117760/117760 [==============================] - 9s 80us/step - loss: 0.6718 - acc: 0.5982 - val_loss: 0.6755 - val_acc: 0.5914
Epoch 6/1000
117760/117760 [==============================] - 9s 78us/step - loss: 0.6718 - acc: 0.5984 - val_loss: 0.6755 - val_acc: 0.5911
Epoch 7/1000
117760/117760 [==============================] - 9s 79us/step - loss: 0.6717 - 

In [0]:
# a=(model.predict(X_train))
# accuracy_score(y_pred=a,y_true=y_test)
# neural_network = KerasClassifier(build_fn=model, 
#                                  epochs=1000, 
#                                  batch_size=100, 
#                                  verbose=0)
# kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
# cross_val_score(model, train, ldf, cv=kfold, scoring="accuracy")
# y_test
# a

In [0]:
# def fold(k):
#   folds = list(StratifiedKFold(n_splits=k, shuffle=True, random_state=1).split(train, ldf))
#   return folds, X_train, y_train

# k = 7
# folds, X_train, y_train = fold(k=7)


In [0]:
# for j, (train_idx, val_idx) in enumerate(folds):
    
#     print('\nFold ',j)
#     X_train_cv = X_train[train_idx]
#     y_train_cv = y_train[train_idx]
#     X_valid_cv = X_train[val_idx]
#     y_valid_cv= y_train[val_idx]
    
# #     name_weights = "final_model_fold" + str(j) + "_weights.h5"
# #     callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)
# #     generator = gen.flow(X_train_cv, y_train_cv, batch_size = batch_size)
# #     model = get_model()
#     model.fit(
# #           generator,
#           steps_per_epoch=len(X_train_cv)/batch_size,
#           epochs=15,
#           shuffle=True,
#           verbose=1,
#           validation_data = (X_valid_cv, y_valid_cv),
#           callbacks = callbacks)
    
#     print(model.evaluate(X_valid_cv, y_valid_cv))